In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from keras.preprocessing.image import ImageDataGenerator
import keras as k
from keras.applications import VGG16
from keras.utils import np_utils
from keras.layers import Dense ,GlobalAveragePooling2D
from keras.models import Model
import sys
from keras import optimizers


In [ ]:
training_dir = '../input/fruits/fruits-360_dataset/fruits-360/Training'
validation_dir = '../input/fruits/fruits-360_dataset/fruits-360/Test'


In [ ]:
image_files = glob(training_dir + '/*/*.jpg')
valid_image_files = glob(validation_dir + '/*/*.jpg')

In [ ]:
folders = glob(training_dir + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))

In [ ]:
training_datagen = ImageDataGenerator(rescale = 1./255)
validation_datagen = ImageDataGenerator(rescale = 1./255)
image_size =(224,224)
training_generator = training_datagen.flow_from_directory(directory = training_dir,target_size=image_size , batch_size=4,class_mode='categorical')
validation_generator = validation_datagen.flow_from_directory(directory = validation_dir,target_size=image_size , batch_size=4,class_mode='categorical')

In [ ]:
training_generator.class_indices

In [ ]:


model = k.applications.VGG16(weights ='imagenet',include_top=False)


output2 = model.output
output2 = GlobalAveragePooling2D()(output2)
output2 = Dense(512,activation='relu')(output2)
predictions = Dense(114,activation='softmax')(output2)
model2 = Model(inputs = model.input,outputs = predictions)


#model = k.applications.MobileNet(input_shape=(96,96,3),weights=None)
model2.summary()

In [ ]:
model2.compile(loss = 'categorical_crossentropy', optimizer = optimizers.SGD(lr=1e-4,momentum=0.9), metrics = ['accuracy'])

In [ ]:
result = model2.fit_generator(training_generator,
                   steps_per_epoch = 57276,   
                   epochs = 1,shuffle=True,verbose=1,  
                   validation_data = validation_generator,
                   validation_steps = 19548)

In [ ]:
plt.plot(result.history['val_acc'])
plt.title('Validation Acc')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

In [ ]:
plt.plot(result.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
test_dir=("../input/fruits/fruits-360_dataset/fruits-360")

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(directory=test_dir,classes=['test-multiple_fruits'],
        target_size=(224,224),
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

In [ ]:
filenames = test_generator.filenames
nb_samples = len(filenames)
nb_samples

In [ ]:
test_generator.reset()
predict = model.predict_generator(test_generator,steps = nb_samples,verbose=1)
